# CellGuide data pipeline prototype

This assumes a local snapshot has been downloaded to the same location as this notebook.

Here is a one-liner for downloading the latest prod snapshot:

```
AWS_PROFILE=profile aws s3 sync s3://bucket-name/$(AWS_PROFILE=profile aws s3 cp s3://bucket-name/latest_snapshot_identifier -)/ local-snapshot-name/
```

**This file should be in the root folder of the `single-cell-data-portal` repo**.

In [1]:
cd ../

/Users/atarashansky/czi/single-cell-data-portal


In [2]:
from backend.wmg.api.v1 import build_filter_dims_values
from backend.wmg.data.ontology_labels import ontology_term_label, ontology_term_id_labels
from backend.wmg.data.snapshot import WmgSnapshot
from backend.wmg.data.query import WmgFiltersQueryCriteria
import tiledb
import pandas as pd
import json
import numpy as np
import os
import openai
from backend.wmg.data.utils import get_datasets_from_curation_api, get_collections_from_curation_api
from backend.wmg.data.rollup import (
    rollup_across_cell_type_descendants,
    rollup_across_cell_type_descendants_array,
    are_cell_types_colinear,
    _descendants,
)
import glob
import requests
import itertools
from scipy import stats
import owlready2
from pronto import Ontology

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
sn = "local-snapshot-name"
snapshot = WmgSnapshot(snapshot_identifier="",
    expression_summary_cube=tiledb.open(f'{sn}/expression_summary'),
    marker_genes_cube=tiledb.open(f'{sn}/marker_genes'),
    expression_summary_default_cube=tiledb.open(f'{sn}/expression_summary_default'),
    expression_summary_fmg_cube=tiledb.open(f'{sn}/expression_summary_fmg'),                       
    cell_counts_cube=tiledb.open(f'{sn}/cell_counts'),
    cell_type_orderings=pd.read_json(f'{sn}/cell_type_orderings.json'),
    primary_filter_dimensions=json.load(open(f'{sn}/primary_filter_dimensions.json','r')),
    dataset_to_gene_ids=json.load(open(f'{sn}/dataset_to_gene_ids.json','r')), 
    filter_relationships=json.load(open(f'{sn}/filter_relationships.json','r')))

In [4]:
ontology = Ontology("https://github.com/obophenotype/cell-ontology/releases/latest/download/cl-basic.obo")
all_cell_types = json.load(open('frontend/src/views/CellGuide/common/fixtures/allCellTypes.json','r'))

# Generate all tissue descriptions

In [ ]:
uberon = Ontology("https://github.com/obophenotype/uberon/releases/download/v2022-08-19/uberon-simple.obo")

In [29]:
classes = [i for i in uberon if i.startswith('UBERON:')]
all_tissue_owl_descriptions = {}
id_to_name = {}
for c in classes :
    c = uberon[c]
    if not c.id.startswith("UBERON:"):
        continue
    if c.obsolete :
        continue
    all_tissue_owl_descriptions[c.id] = str(c.definition) if str(c.definition) != 'None' else ''

In [34]:
cc = snapshot.cell_counts_cube.df[:]

In [35]:
all_relevant_tissues = list(set(list(cc['tissue_ontology_term_id'].unique())+list(cc['tissue_original_ontology_term_id'].unique())))

In [36]:
all_tissue_owl_descriptions_filt = {i: all_tissue_owl_descriptions[i] for i in all_tissue_owl_descriptions if i in all_relevant_tissues}

## Generate source data per cell type

In [13]:
%env DEPLOYMENT_STAGE=prod
%env API_URL=https://api.cellxgene.cziscience.com

def get_title_from_doi(doi):
    url = f"https://api.crossref.org/works/{doi}"

    # Send a GET request to the API
    response = requests.get(url)

    # If the GET request is successful, the status code will be 200
    if response.status_code == 200:
        # Get the response data
        data = response.json()

        # Get the title and citation count from the data
        try:
            title = data['message']['title'][0]
        except:
            try:
                title = data['message']['items'][0]['title'][0]
            except:
                return doi
        return title
    else:
        return doi
    
def format_citation(metadata):
    first_author = metadata['publisher_metadata']['authors'][0]
    if "family" in first_author:
        author_str = f"{first_author['family']} et al."
    else:
        author_str = f"{first_author['name']} et al."
    
    journal = metadata['publisher_metadata']['journal']
    year = metadata['publisher_metadata']['published_year']
    
    return f"{author_str} ({year}) {journal}"

env: DEPLOYMENT_STAGE=prod
env: API_URL=https://api.cellxgene.cziscience.com


In [11]:
snapshot.build_dataset_metadata_dict()

datasets = get_datasets_from_curation_api()
collections = get_collections_from_curation_api()

collections_dict = {collection['collection_id']: collection for collection in collections}
datasets_dict = {dataset['dataset_id']: dataset for dataset in datasets}

env: DEPLOYMENT_STAGE=prod
env: API_URL=https://api.cellxgene.cziscience.com


In [14]:
cts = [i['id'] for i in all_cell_types]

DATA = {}

for i in cts:
    seen_datasets = []
    lineage =_descendants(i)    
    assert i in lineage
    datasets=[]
    for d in datasets_dict:
        for c in datasets_dict[d]['cell_type']:
            if c['ontology_term_id'] in lineage:
                if d not in seen_datasets and datasets_dict
    for organism in snapshot.primary_filter_dimensions['tissue_terms']:    
        criteria = WmgFiltersQueryCriteria(organism_ontology_term_id=organism,
                                           cell_type_ontology_term_ids=lineage)
        res = build_filter_dims_values(criteria, snapshot)
        data = res['datasets']
        for datum in data:
            if datum['id'] not in seen_datasets and datum['collection_id']!='':
                seen_datasets.append(datum['id'])
                datasets.append(datum)

    collections_to_datasets = {}
    for dataset in datasets:
        dataset = datasets_dict[dataset['dataset_id']]
        
        a = collections_to_datasets.get(dataset['collection_id'],{})

        a['collection_name'] = collections_dict[dataset['collection_id']]['name']
        a['collection_url'] = collections_dict[dataset['collection_id']]['collection_url']
        a['publication_url'] = collections_dict[dataset['collection_id']]['doi']
        if collections_dict[dataset['collection_id']]['publisher_metadata']:
            a['publication_title'] = format_citation(collections_dict[dataset['collection_id']])
        else:
            a['publication_title'] = "No publication"
            
        tissues = a.get("tissue", [])
        diseases = a.get("disease", [])
        organisms = a.get("organism", [])
        for tissue in dataset['tissue']:
            if tissue['ontology_term_id'] not in [i['ontology_term_id'] for i in tissues]:
                tissues.append(tissue)
            
        for disease in dataset['disease']:
            if disease['ontology_term_id'] not in [i['ontology_term_id'] for i in diseases]:
                diseases.append(disease)
            
        for organism in dataset['organism']:
            if organism['ontology_term_id'] not in [i['ontology_term_id'] for i in organisms]:
                organisms.append(organism)
            
        a['tissue'] = tissues
        a['disease'] = diseases
        a['organism'] = organisms
    
        collections_to_datasets[dataset['collection_id']]=a

    DATA[i] = list(collections_to_datasets.values())                                 

## Generate enriched genes and expression metrics per cell type

In [123]:
import time
from numba import njit, prange
import numpy as np

@njit(parallel=True)
def nanpercentile_2d(arr, percentile, axis):
    """
    Calculate the specified percentile of a 2D array along an axis, ignoring NaN values.

    Parameters:
        arr: 2D array to calculate percentile of
        percentile: percentile to calculate, as a number between 0 and 100
        axis: axis along which to calculate percentile

    Returns:
        The specified percentile of the 2D array along the specified axis.
    """
    if axis == 0:
        result = np.empty(arr.shape[1])
        for i in prange(arr.shape[1]):
            arr_column = arr[:, i]
            result[i] = nanpercentile(arr_column, percentile)
        return result
    else:
        result = np.empty(arr.shape[0])
        for i in prange(arr.shape[0]):
            arr_row = arr[i, :]
            result[i] = nanpercentile(arr_row, percentile)
        return result

@njit
def nanpercentile(arr, percentile):
    arr_without_nan = arr[np.logical_not(np.isnan(arr))]
    length = len(arr_without_nan)

    if length == 0:
        return np.nan

    return np.percentile(arr_without_nan,percentile)

def _run_ttest(sum1, sumsq1, n1, sum2, sumsq2, n2):
    with np.errstate(divide="ignore", invalid="ignore"):
        mean1 = sum1 / n1
        meansq1 = sumsq1 / n1

        mean2 = sum2 / n2
        meansq2 = sumsq2 / n2

        var1 = meansq1 - mean1**2
        var1[var1 < 0] = 0
        var2 = meansq2 - mean2**2
        var2[var2 < 0] = 0

        var1_n = var1 / n1
        var2_n = var2 / n2
        sum_var_n = var1_n + var2_n
        dof = sum_var_n**2 / (var1_n**2 / (n1 - 1) + var2_n**2 / (n2 - 1))
        tscores = (mean1 - mean2) / np.sqrt(sum_var_n)
        effects = (mean1 - mean2) / np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 1))

    pvals = stats.t.sf(tscores, dof)
    return pvals, effects

def _post_process_stats(
    cell_type_target,
    cell_types_context,
    genes,
    pvals,
    effects,
    percentile=0.05
):
    is_colinear = np.array(
        [are_cell_types_colinear(cell_type, cell_type_target) for cell_type in cell_types_context]
    )
    effects[is_colinear] = np.nan
    pvals[is_colinear] = np.nan
    
    pvals[:,np.all(np.isnan(pvals),axis=0)]=1
    effects[:,np.all(np.isnan(effects),axis=0)]=0

    # aggregate
    effects = nanpercentile_2d(effects, percentile * 100, 0)
    
    effects[effects==0]=np.nan
    
    # pvals = np.array([stats.combine_pvalues(x[np.invert(np.isnan(x))] + 1e-300)[-1] for x in pvals.T])
    pvals = np.sort(pvals,axis=0)[int(np.round(0.05*pvals.shape[0]))]
    
    markers = np.array(genes)[np.argsort(-effects)]
    p = pvals[np.argsort(-effects)]
    effects = effects[np.argsort(-effects)]
    
    statistics = []
    final_markers = []
    for i in range(len(p)):
        pi = p[i]
        ei = effects[i]
        if ei is not np.nan and pi is not np.nan:
            statistics.append({f"p_value": pi, f"effect_size": ei})
            final_markers.append(markers[i])
    return dict(zip(list(final_markers), statistics))

In [18]:
organism_id_to_name = {}
[organism_id_to_name.update(i) for i in snapshot.primary_filter_dimensions['organism_terms']];

cell_counts_df = snapshot.cell_counts_cube.df[:]
cell_counts_df = cell_counts_df.groupby(['organism_ontology_term_id','cell_type_ontology_term_id']).sum(numeric_only=True)

expressions_df = snapshot.expression_summary_fmg_cube.df[:]
expressions_df = expressions_df.groupby(['organism_ontology_term_id','cell_type_ontology_term_id','gene_ontology_term_id']).sum(numeric_only=True)
expressions_df = expressions_df.reset_index()
expressions_df = expressions_df[expressions_df['cell_type_ontology_term_id'].isin(all_cell_type_ids)]

all_cell_type_ids = [i['id'] for i in all_cell_types]
cell_counts_df = cell_counts_df[cell_counts_df.index.get_level_values('cell_type_ontology_term_id').isin(all_cell_type_ids)]

index = pd.Index(list(itertools.product(cell_counts_df.index.get_level_values('organism_ontology_term_id').unique(),all_cell_type_ids)))
index = index.set_names(['organism_ontology_term_id','cell_type_ontology_term_id'])
universe_cell_counts_df = pd.DataFrame(index = index)
universe_cell_counts_df['n_cells']=0
universe_cell_counts_df['n_cells'][cell_counts_df.index] = cell_counts_df['n_cells']

universe_cell_counts_df = rollup_across_cell_type_descendants(
    universe_cell_counts_df.reset_index()
)
universe_cell_counts_df=universe_cell_counts_df[universe_cell_counts_df['n_cells']>0]
universe_cell_counts_df = universe_cell_counts_df.groupby(['organism_ontology_term_id','cell_type_ontology_term_id']).sum()

In [19]:
a = list(set(list(universe_cell_counts_df.index.get_level_values('cell_type_ontology_term_id'))))

In [20]:
all_cell_types = [c for c in all_cell_types if c['id'] in a]

In [ ]:
x = list(zip(*expressions_df[['organism_ontology_term_id','cell_type_ontology_term_id']].values.T))
y = list(expressions_df['gene_ontology_term_id'])

xu = list(set(x))
yu = list(set(y))

x_index = pd.Series(index=pd.Index(xu),data=np.arange(len(xu)))
y_index = pd.Series(index=pd.Index(yu),data=np.arange(len(yu)))

e_nnz = np.zeros((len(xu),len(yu)))
e_sum = np.zeros((len(xu),len(yu)))
e_sqsum = np.zeros((len(xu),len(yu)))

e_nnz[x_index[x].values, y_index[y].values] = expressions_df['nnz']
e_sum[x_index[x].values, y_index[y].values] = expressions_df['sum']
e_sqsum[x_index[x].values, y_index[y].values] = expressions_df['sqsum']

available_combinations = set(list(universe_cell_counts_df.index.values))
missing_combinations = available_combinations.difference(xu)

xu = xu + list(missing_combinations)
x_index = pd.Series(index=pd.Index(xu),data=np.arange(len(xu)))

e_nnz = np.vstack((e_nnz,np.zeros((len(missing_combinations),e_nnz.shape[1]))))
e_sum = np.vstack((e_sum,np.zeros((len(missing_combinations),e_sum.shape[1]))))
e_sqsum = np.vstack((e_sqsum,np.zeros((len(missing_combinations),e_sqsum.shape[1]))))

organisms = x_index.index.get_level_values(0)
organisms_u = list(set(organisms))
cell_types = x_index.index.get_level_values(1)

e_nnz_rollup = np.zeros_like(e_nnz)
e_sum_rollup = np.zeros_like(e_sum)
e_sqsum_rollup = np.zeros_like(e_sqsum)

n_cells = universe_cell_counts_df['n_cells'][x_index.index]
n_cells = np.tile(n_cells.values[:,None],(1,e_nnz_rollup.shape[1]))

In [124]:
all_results = []
for organism in organisms_u:
    cell_types_o = cell_types[organisms==organism]
    e_nnz_o = e_nnz[organisms==organism]
    e_sum_o = e_sum[organisms==organism]    
    e_sqsum_o = e_sqsum[organisms==organism]        
    n_cells_o = n_cells[organisms==organism]
    
    e_nnz_o = rollup_across_cell_type_descendants_array(e_nnz_o,cell_types_o)
    e_sum_o = rollup_across_cell_type_descendants_array(e_sum_o,cell_types_o)  
    e_sqsum_o = rollup_across_cell_type_descendants_array(e_sqsum_o,cell_types_o)  
    
    e_nnz_rollup[organisms==organism]=e_nnz_o
    e_sum_rollup[organisms==organism]=e_sum_o
    e_sqsum_rollup[organisms==organism]=e_sqsum_o 
    
    i_range = np.arange(e_sum_o.shape[0])
    for i in range(e_sum_o.shape[0]):
        print(i)
        sum1 = e_sum_o[i][None,:].copy()
        sumsq1 = e_sqsum_o[i][None,:].copy()
        n1 = n_cells_o[i][None,:].copy()


        pvals, effects = _run_ttest(sum1,sumsq1,n1,
                   e_sum_o, e_sqsum_o, n_cells_o)

        pvals[i] = np.nan
        effects[i] = np.nan
     
        res = _post_process_stats(
            cell_types_o[i],
            cell_types_o,
            y_index.index.values,
            pvals,
            effects,
            percentile=0.05
        )

        res2 = pd.DataFrame()
        res2.index = pd.Index([k for k in res])
        res2['p_value'] = [res[k]['p_value'] for k in res]
        res2['effect_size'] = [res[k]['effect_size'] for k in res]
        res = res2     

        res['cell_type_ontology_term_id'] = cell_types_o[i]
        res['organism_ontology_term_id'] = organism
        res['gene_ontology_term_id'] = res.index
        res = res.reset_index(drop=True)
        res = res[res['effect_size'].notnull()]
        res = res[res['effect_size']>0]        
        all_results.append(res)
        
x_new,y_new = (e_nnz_rollup+e_sum_rollup).nonzero()

r_x_index = pd.Series(index=x_index.values,data=x_index.index.values)
r_y_index = pd.Series(index=y_index.values,data=y_index.index.values)

y_r_new = r_y_index[y_new].values

x_r_new = r_x_index[x_new].values

new_index = pd.Index([i+(j,) for i,j in zip(x_r_new,y_r_new)])

nnz_flat = e_nnz_rollup[x_new,y_new]
sum_flat = e_sum_rollup[x_new,y_new]

new_index = new_index.set_names(['organism_ontology_term_id','cell_type_ontology_term_id','gene_ontology_term_id'])
new_expression_rollup = pd.DataFrame(index=new_index)

new_expression_rollup['nnz']=nnz_flat
new_expression_rollup['sum']=sum_flat

new_expression_rollup=new_expression_rollup.reset_index()

markers_df = pd.concat(all_results,axis=0)
markers_df=markers_df[markers_df['p_value']<1e-5]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
23

In [146]:
dims = ["organism_ontology_term_id","cell_type_ontology_term_id","gene_ontology_term_id"]
markers_df = markers_df.groupby(dims).first()
new_expression_rollup = new_expression_rollup.groupby(dims).first()

In [155]:
new_expression_rollup = new_expression_rollup.join(markers_df,on=dims)

In [158]:
new_expression_rollup = new_expression_rollup.reset_index()
markers_df = markers_df.reset_index()

In [ ]:
top_per_group = markers_df.groupby(['organism_ontology_term_id','cell_type_ontology_term_id']).apply(lambda x: x.nlargest(100, 'effect_size'))

columns = ['organism_ontology_term_id','cell_type_ontology_term_id','gene_ontology_term_id']
o_ct_genes = list(zip(*top_per_group[columns].values.T))


filt1 = new_expression_rollup['cell_type_ontology_term_id'].isin(top_per_group['cell_type_ontology_term_id'].unique())

filt2 = new_expression_rollup['gene_ontology_term_id'].isin(top_per_group['gene_ontology_term_id'].unique())

filt = np.logical_and(filt1,filt2)

new_expression_rollup = new_expression_rollup[filt]

new_expression_rollup.index = pd.Index(list(zip(*new_expression_rollup[['organism_ontology_term_id','cell_type_ontology_term_id','gene_ontology_term_id']].values.T)))

universe_cell_counts_df = universe_cell_counts_df.groupby(['organism_ontology_term_id','cell_type_ontology_term_id']).sum()['n_cells']

gene_id_to_symbol={}
all_genes = []
for k in snapshot.primary_filter_dimensions['gene_terms']:
    for i in snapshot.primary_filter_dimensions['gene_terms'][k]:
        gene_id_to_symbol.update(i)
        all_genes.append(list(i.keys())[0])

gene_id_to_name = pd.read_csv('ensembl_gene_ids_to_descriptions.tsv.gz',sep='\t')

gene_id_to_name = gene_id_to_name.set_index('Ensembl GeneIDs')['Description'].to_dict()

data={}
for i in o_ct_genes:
    o,ct,gene = i
    
    nnz = new_expression_rollup['nnz'][i]
    s = new_expression_rollup['sum'][i]
    es = new_expression_rollup['effect_size'][i]
    n_cells = universe_cell_counts_df[(o,ct)]
    
    a = data.get(ct,[])
    a.append({
        'me': s/nnz if nnz > 0 else 0,
        'pc': nnz/n_cells,
        'marker_score': es,
        'symbol': gene_id_to_symbol[gene],
        'name': gene_id_to_name.get(gene,gene_id_to_symbol[gene]),
        'organism': organism_id_to_name[o]
    })
    data[ct]=a